In [106]:
from pystac_client import Client
from odc.stac import load
import odc.geo
import xarray as xr
import rioxarray as rio
import geopandas as gpd 
import itertools
from multiprocessing import Pool
import dask.distributed
import dask.utils
from odc.stac import configure_rio, stac_load
from odc.algo import erase_bad, mask_cleanup
import numpy as np

CLOUD_SHADOWS = 3
CLOUD_HIGH_PROBABILITY = 9
NO_DATA = 0

bitmask_cloud = 0
for field in [CLOUD_SHADOWS, CLOUD_HIGH_PROBABILITY]:
    bitmask_cloud |= 1 << field

bitmask_nodata = 0
for field in [NO_DATA]:
    bitmask_nodata |= 1 << field


# Start dask cluster
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client = client)

# Prepare region of interest
spain = gpd.read_file('/Users/diegobengochea/git/iberian.carbon/data/SpainPolygon/gadm41_ESP_1.shp')
#spain = gpd.read_file('/home/dibepa/git/iberian.carbon/spain/gadm41_ESP_1.shp')
# Filter continental Spain
spain = spain[ (spain.GID_1 != 'ESP.7_1') & (spain.GID_1 != 'ESP.13_1') & (spain.GID_1 != 'ESP.14_1') ] 
# Reproject to UTM30 EPSG:25830
spain = spain.dissolve()[['geometry','COUNTRY']].to_crs(epsg='25830')
# Add CRS information to shapely polygon
geometry_spain = odc.geo.geom.Geometry(spain.geometry[0],crs='EPSG:25830')

# Create a GeoBox for all continental Spain with a 10 meters resolution 
geobox_spain = odc.geo.geobox.GeoBox.from_geopolygon(geometry_spain,resolution=10) # The resolution here is irrelevant since the Spain GeoBOX is too large to make queries, adn thus cannot be used as intersect. Only used to create tiles of suitable shape 20km2



/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60764 instead
  warnings.warn(


2024-09-24 19:16:52,715 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 20 memory: 1727 MB fds: 351>>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/tornado/ioloop.py", line 937, in _run
    val = self.callback()
          ^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/distributed/system_monitor.py", line 168, in update
    net_ioc = psutil.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/psutil/__init__.py", line 2139, in net_io_counters
    rawdict = _psplatform.net_io_counters()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 12] Cannot allocate memory
2024-09-24 21:58:23,433 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor

In [107]:
# Divide the full geobox in Geotiles of smaller size for processing
geotiles_spain = odc.geo.geobox.GeoboxTiles(geobox_spain,(100,100))

geotiles_spain = [ geotiles_spain.__getitem__(tile) for tile in geotiles_spain._all_tiles() ]

In [17]:
len(geotiles_spain)

914788

In [108]:
geobox = geotiles_spain[110000]
year = 2018

bbox = geobox.boundingbox.to_crs('EPSG:4326')
bbox = (bbox.left,bbox.bottom,bbox.right,bbox.top)

green_season = f'{year}-04-01/{year}-05-01'

catalog = Client.open("https://earth-search.aws.element84.com/v1") 
search = catalog.search(
    collections = ['sentinel-2-l2a'],
    bbox = bbox, 
    datetime = green_season,
    query = ['eo:cloud_cover<50']
)

item_collection = search.item_collection()
print(len(item_collection))



2


In [19]:
geobox.boundingbox.to_crs('epsg:4326')

BoundingBox(left=-2.6329613682457893, bottom=43.01382645884041, right=-2.62063475983079, top=43.02287134535306, crs=CRS('EPSG:4326'))

In [194]:
abs(int(geobox.resolution.y))

10

In [55]:
item_collection.save_object('sentinel2-items.json')

In [195]:
src_dataset = odc.stac.load(
    item_collection,
    bands = ['red', 'green', 'blue', 'nir', 'swir16', 'swir22', 'rededge1', 'rededge2', 'rededge3', 'nir08','scl'],
    geobox = geobox,
    chunks = {'x':10,'y':10},
    groupby = 'solar_day',
    resampling = 'bilinear'
)

#src_dataset[['red', 'green', 'blue', 'nir', 'swir16', 'swir22', 'rededge1', 'rededge2', 'rededge3', 'nir08']] = src_dataset[['red', 'green', 'blue', 'nir', 'swir16', 'swir22', 'rededge1', 'rededge2', 'rededge3', 'nir08']].astype('float32')

cloud_mask = src_dataset.scl.astype("uint16") & bitmask_cloud != 0
cloud_mask = mask_cleanup(cloud_mask) # Use default filters
src_dataset = erase_bad(src_dataset, cloud_mask)

src_dataset = src_dataset.where(~cloud_mask)
src_dataset = src_dataset.where(~nodata_mask)

src_dataset

<xarray.Dataset> Size: 882kB
Dimensions:      (time: 2, y: 100, x: 100)
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 16B 2018-04-20T11:26:11.293000 2018-04...
Data variables:
    red          (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    green        (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    blue         (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    nir          (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    swir16       (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    swir22       (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    rededge1     (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    rededge2     (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    rededge3     (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    nir08        (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    scl          (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

In [199]:
data = src_dataset[['blue','scl']]
data

<xarray.Dataset> Size: 162kB
Dimensions:      (time: 2, y: 100, x: 100)
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
  * time         (time) datetime64[ns] 16B 2018-04-20T11:26:11.293000 2018-04...
Data variables:
    blue         (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>
    scl          (time, y, x) float32 80kB dask.array<chunksize=(1, 10, 10), meta=np.ndarray>

In [200]:
med = data.median(dim='time')
medskip = data.median(dim='time',skipna=True)

In [201]:
med = med.astype('uint16')

In [202]:
for var in med.variables:
    med[var].rio.write_nodata(0,inplace=True)

med

<xarray.Dataset> Size: 42kB
Dimensions:      (y: 100, x: 100)
Coordinates:
  * y            (y) float64 800B 4.753e+06 4.753e+06 ... 4.752e+06 4.752e+06
  * x            (x) float64 800B 1.699e+05 1.699e+05 ... 1.709e+05 1.709e+05
    spatial_ref  int32 4B 25830
Data variables:
    blue         (y, x) uint16 20kB dask.array<chunksize=(100, 100), meta=np.ndarray>
    scl          (y, x) uint16 20kB dask.array<chunksize=(100, 100), meta=np.ndarray>

In [208]:
import threading

from dask.distributed import Client, LocalCluster, Lock
med = med.compute()
med.rio.to_raster(
    f'/Users/diegobengochea/git/iberian.carbon/annual_sentinel_mosaics/raster_nodata_test.tif',
    tags = {'DATETIME':'2028'},
    **{'compress': 'lzw'},
    tiled = True,
    #compute=True,
    lock=threading.Lock()
)
!rio info raster_nodata_test.tif

{"blockxsize": 256, "blockysize": 256, "bounds": [169910.0, 4752420.0, 170910.0, 4753420.0], "colorinterp": ["gray", "undefined"], "compress": "lzw", "count": 2, "crs": "EPSG:25830", "descriptions": ["blue", "scl"], "driver": "GTiff", "dtype": "uint16", "height": 100, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-7.034089023446267, 42.85777464674912], "mask_flags": [["nodata"], ["nodata"]], "nodata": 0.0, "res": [10.0, 10.0], "shape": [100, 100], "tiled": true, "transform": [10.0, 0.0, 169910.0, 0.0, -10.0, 4753420.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 100}


In [183]:
ge



{"blockysize": 20, "bounds": [169910.0, 4752420.0, 170910.0, 4753420.0], "colorinterp": ["gray", "undefined"], "compress": "lzw", "count": 2, "crs": "EPSG:25830", "descriptions": ["blue", "scl"], "driver": "GTiff", "dtype": "uint16", "height": 100, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-7.034089023446267, 42.85777464674912], "mask_flags": [["nodata"], ["nodata"]], "nodata": 0.0, "res": [10.0, 10.0], "shape": [100, 100], "tiled": false, "transform": [10.0, 0.0, 169910.0, 0.0, -10.0, 4753420.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 100}


In [97]:
import numpy as np
for time in range(5):
    print(np.unique(data.blue[time]))

[   0   71   74 ... 3217 3300 3362]
[   0  123  130 ... 2991 3127 3142]


IndexError: index 2 is out of bounds for axis 0 with size 2

In [16]:
median = src_dataset['red'].median(dim='time',skipna=True).compute().fillna(0).astype('uint16')


/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.12/site-packages/rasterio/warp.py:344: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _reproject(
/opt/homebrew/Caskroom/miniforge/base/envs/sentinel.mosaics/lib/python3.

<xarray.DataArray 'red' (y: 100, x: 100)> Size: 20kB
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)
Coordinates:
  * y            (y) float64 800B 4.859e+06 4.859e+06 ... 4.858e+06 4.858e+06
  * x            (x) float64 800B -1.408e+04 -1.408e+04 ... -1.31e+04 -1.31e+04
    spatial_ref  int32 4B 25830